# Carbon Reduction Experiment Analysis

**Empirical Assessment of Carbon Reduction Strategies in Enterprise Software Applications**

This notebook presents the statistical analysis of controlled experiments investigating the impact of application-level configurations on carbon emissions in AFAS SB. The analysis addresses Research Questions 2 and 3 from the thesis, employing statistical methods to evaluate configuration impacts and optimization potential.

## Research Questions

**RQ2**: How do application-level configuration settings affect the rate of carbon emissions of AFAS SB?

**RQ3**: To what extent can the rate of carbon emissions of AFAS SB be reduced by _combining_ application-level configurations while maintaining system performance and functionality?

## Hypotheses

**H1.0**: Application-level configuration changes do not significantly affect the SCI score  
**H1.a**: At least one configuration change significantly affects the SCI score  

**H2.0**: The combined optimal SCI score configuration does not significantly affect the SCI score  
**H2.a**: The combined optimal SCI score configuration significantly affects the SCI score

## Setup and Data Loading

Loading experimental data from the controlled environment.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import scipy.stats as stats
from scipy.stats import shapiro, levene, mannwhitneyu, kruskal, ttest_ind, f_oneway
from scipy.stats import pearsonr, spearmanr, bootstrap
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.power import ttest_power
from statsmodels.stats.contingency_tables import mcnemar
from statsmodels.stats.power import tt_solve_power
from pathlib import Path
import warnings
from typing import Dict, List, Tuple
from matplotlib.patches import Rectangle
import os

warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams.update({
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.figsize': (10, 6),
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight'
})

figures_dir = Path('figures')
figures_dir.mkdir(exist_ok=True)
results_dir = Path('results_data')
results_dir.mkdir(exist_ok=True)

print("Analysis environment initialized successfully")

## SCI Calculator and Data Loading Functions

Implementing the Software Carbon Intensity calculations according to Green Software Foundation specification and data loading utilities.

In [ ]:
class SCICalculator:
    
    def __init__(self):
        self.GRID_CARBON_INTENSITY = 272  # gCO2eq/kWh, April 2025, The Netherlands
        self.TOTAL_EMBODIED_EMISSIONS = 1449.84  # kg CO2eq
        self.HARDWARE_LIFESPAN_HOURS = 4 * 365 * 24  # 4 years
        self.VM_CPU_CORES = 8
        self.TOTAL_SERVER_CORES = 24
        
    def calculate_sci(self, power_watts: float, duration_seconds: float, 
                     functional_units: int) -> Dict[str, float]:
        
        # Operational Emissions (O = E × I)
        energy_kwh = (power_watts * duration_seconds) / (1000 * 3600)
        operational_emissions = energy_kwh * self.GRID_CARBON_INTENSITY
        
        # Embodied Emissions (M = TE × (TiR/EL) × (RR/ToR))
        time_reserved_hours = duration_seconds / 3600
        resource_ratio = self.VM_CPU_CORES / self.TOTAL_SERVER_CORES
        
        embodied_emissions = (
            self.TOTAL_EMBODIED_EMISSIONS * 1000 *
            (time_reserved_hours / self.HARDWARE_LIFESPAN_HOURS) *
            resource_ratio
        )
        
        # SCI Score = (O + M) / R
        total_emissions = operational_emissions + embodied_emissions
        sci_score = total_emissions / functional_units if functional_units > 0 else 0
        
        return {
            'energy_kwh': energy_kwh,
            'operational_emissions': operational_emissions,
            'embodied_emissions': embodied_emissions,
            'total_emissions': total_emissions,
            'sci_score': sci_score
        }

def load_experiment_data(base_path: str = "experiment_runner/afas-sb/experiments") -> Dict[str, pd.DataFrame]:
    sci_calc = SCICalculator()
    data = {}
    
    experiments = {
        'P1_E1': {'name': 'Baseline Configuration', 'category': 'baseline'},
        'P2_E1': {'name': 'Parallelism Configuration', 'category': 'parallelism', 'factor': 'parallelism_config'},
        'P2_E2': {'name': 'Logging Configuration', 'category': 'logging', 'factor': 'logging_config'},
        'P2_E3': {'name': 'Cache Configuration', 'category': 'caching', 'factor': 'cache_config'},
        'P2_E4': {'name': 'Compression Configuration', 'category': 'compression', 'factor': 'compression_config'},
        'P2_E5': {'name': 'Garbage Collection Configuration', 'category': 'gc', 'factor': 'gc_config'},
        'P3_E1': {'name': 'Carbon-Optimized Configuration', 'category': 'optimized', 'factor': 'optimization_config'}
    }
    
    for exp_id in experiments.keys():
        run_table_path = Path(base_path) / exp_id / 'run_table.csv'
        
        if run_table_path.exists():
            df = pd.read_csv(run_table_path)
            df = df[df['__done'] == 'DONE'].copy()
            
            if len(df) > 0:
                sci_results = df.apply(lambda row: sci_calc.calculate_sci(
                    power_watts=row.get('powerjoular_power', 0),
                    duration_seconds=row.get('test_duration', 0),
                    functional_units=row.get('scenario_count', 1)
                ), axis=1)
                
                for key in ['energy_kwh', 'operational_emissions', 'embodied_emissions', 'total_emissions', 'sci_score']:
                    df[key] = [result[key] for result in sci_results]
                
                df['experiment_id'] = exp_id
                df['experiment_name'] = experiments[exp_id]['name']
                df['category'] = experiments[exp_id]['category']
                
                if exp_id == 'P3_E1' and 'optimization_config' in df.columns:
                    df = df[df['optimization_config'] == 'carbon_optimized'].copy()
                
                data[exp_id] = df
                print(f"Loaded {exp_id}: {len(df)} runs, mean SCI: {df['sci_score'].mean():.3f} gCO2eq/scenario")
    
    return data, experiments

experiment_data, experiments = load_experiment_data()
all_data = pd.concat(experiment_data.values(), ignore_index=True)
print(f"\nTotal runs loaded: {len(all_data)}")
print(f"Overall mean SCI: {all_data['sci_score'].mean():.3f} gCO2eq/scenario")

## Run Duration Analysis

Analyzing the execution time characteristics of all experimental runs.

In [ ]:
def calculate_duration_statistics(experiment_data):
    all_durations = []
    duration_by_experiment = {}
    
    for exp_id, data in experiment_data.items():
        if 'test_duration' in data.columns:
            exp_durations = data['test_duration'].dropna()
            all_durations.extend(exp_durations.tolist())
            
            duration_by_experiment[exp_id] = {
                'count': len(exp_durations),
                'mean': exp_durations.mean(),
                'std': exp_durations.std(),
                'median': exp_durations.median(),
                'min': exp_durations.min(),
                'max': exp_durations.max(),
                'total': exp_durations.sum()
            }
    
    all_durations = pd.Series(all_durations)
    
    duration_stats = {
        'total_runs': len(all_durations),
        'total_duration_seconds': all_durations.sum(),
        'total_duration_minutes': all_durations.sum() / 60,
        'total_duration_hours': all_durations.sum() / 3600,
        'mean_duration': all_durations.mean(),
        'std_duration': all_durations.std(),
        'median_duration': all_durations.median(),
        'min_duration': all_durations.min(),
        'max_duration': all_durations.max(),
        'percentile_25': all_durations.quantile(0.25),
        'percentile_75': all_durations.quantile(0.75),
        'by_experiment': duration_by_experiment
    }
    
    return duration_stats

duration_stats = calculate_duration_statistics(experiment_data)

print("=== RUN DURATION ANALYSIS ===\n")

print(f"Total number of runs: {duration_stats['total_runs']}")
print(f"Total duration of all runs: {duration_stats['total_duration_seconds']:.1f} seconds")
print(f"Total duration of all runs: {duration_stats['total_duration_minutes']:.1f} minutes")
print(f"Total duration of all runs: {duration_stats['total_duration_hours']:.2f} hours")
print(f"\nMean run duration: {duration_stats['mean_duration']:.1f} seconds")
print(f"Median run duration: {duration_stats['median_duration']:.1f} seconds")
print(f"Standard deviation: {duration_stats['std_duration']:.1f} seconds")
print(f"Min run duration: {duration_stats['min_duration']:.1f} seconds")
print(f"Max run duration: {duration_stats['max_duration']:.1f} seconds")
print(f"25th percentile: {duration_stats['percentile_25']:.1f} seconds")
print(f"75th percentile: {duration_stats['percentile_75']:.1f} seconds")

print(f"\n=== DURATION BY EXPERIMENT ===")
for exp_id, stats in duration_stats['by_experiment'].items():
    print(f"\n{exp_id}:")
    print(f"  Runs: {stats['count']}")
    print(f"  Total duration: {stats['total']:.1f} seconds ({stats['total']/60:.1f} minutes)")
    print(f"  Mean: {stats['mean']:.1f} ± {stats['std']:.1f} seconds")
    print(f"  Range: {stats['min']:.1f} - {stats['max']:.1f} seconds")

duration_summary_df = pd.DataFrame([
    {
        'Experiment': exp_id,
        'Run_Count': stats['count'],
        'Total_Duration_Minutes': stats['total'] / 60,
        'Mean_Duration_Seconds': stats['mean'],
        'Std_Duration_Seconds': stats['std'],
        'Min_Duration_Seconds': stats['min'],
        'Max_Duration_Seconds': stats['max']
    }
    for exp_id, stats in duration_stats['by_experiment'].items()
])

print(f"\n=== DURATION SUMMARY TABLE ===")
print(duration_summary_df.round(1).to_string(index=False))

## Individual Test Scenario Analysis

Analyzing the characteristics of individual test scenarios within each experimental run to understand the granular workload composition and execution patterns.

In [ ]:
def get_scenario_stats():
    print("=== TEST SCENARIO ANALYSIS ===\n")
    
    import glob
    scenario_files = glob.glob("experiment_runner/afas-sb/experiments/*/*/scenario_summary.csv")
    
    if scenario_files:
        all_scenarios = []
        valid_files = 0
        
        for file_path in scenario_files:
            try:
                df = pd.read_csv(file_path)
                if not df.empty:
                    all_scenarios.append(df)
                    valid_files += 1
            except:
                continue
        
        if all_scenarios:
            combined_scenarios = pd.concat(all_scenarios, ignore_index=True)
            
            print(f"Analysis across ALL experimental runs:")
            print(f"Total scenario files analyzed: {valid_files}")
            print(f"Total individual scenario executions: {len(combined_scenarios)}")
            print(f"Scenarios per run: {len(combined_scenarios) // valid_files}")
            
            if 'duration' in combined_scenarios.columns:
                durations = combined_scenarios['duration'].dropna() / 1000
                print(f"\nScenario Duration Statistics:")
                print(f"  Mean duration: {durations.mean():.1f} seconds")
                print(f"  Median duration: {durations.median():.1f} seconds")
                print(f"  Standard deviation: {durations.std():.1f} seconds")
                print(f"  Duration range: {durations.min():.1f} - {durations.max():.1f} seconds")
                print(f"  25th percentile: {durations.quantile(0.25):.1f} seconds")
                print(f"  75th percentile: {durations.quantile(0.75):.1f} seconds")
            
            if 'status' in combined_scenarios.columns:
                success_rate = (combined_scenarios['status'] == 'Succeeded').mean() * 100
                failed_scenarios = len(combined_scenarios[combined_scenarios['status'] != 'Succeeded'])
                print(f"\nReliability Statistics:")
                print(f"  Overall success rate: {success_rate:.1f}%")
                print(f"  Failed scenarios: {failed_scenarios} out of {len(combined_scenarios)}")
        else:
            print("No valid scenario data could be loaded")
    else:
        print("No scenario summary files found")

get_scenario_stats()

## Data Preprocessing and Baseline Reference

Establishing the baseline reference (P1_E1) for comparative analysis and preparing configuration-specific datasets for RQ2 analysis.

In [ ]:
baseline_data = experiment_data['P1_E1']
baseline_sci_mean = baseline_data['sci_score'].mean()
baseline_sci_std = baseline_data['sci_score'].std()

print(f"Baseline SCI: {baseline_sci_mean:.3f} ± {baseline_sci_std:.3f} gCO2eq/scenario (n={len(baseline_data)})")

config_experiments = {
    'P2_E1': 'parallelism_config',
    'P2_E2': 'logging_config',
    'P2_E3': 'cache_config', 
    'P2_E5': 'gc_config'
}

config_data = {}
for exp_id, config_column in config_experiments.items():
    if exp_id in experiment_data and config_column in experiment_data[exp_id].columns:
        exp_df = experiment_data[exp_id]
        configs = exp_df[config_column].unique()
        
        for config in configs:
            config_df = exp_df[exp_df[config_column] == config].copy()
            config_key = f"{exp_id}_{config}"
            config_data[config_key] = config_df
            print(f"{config_key}: {len(config_df)} runs")

if 'P2_E4' in experiment_data:
    config_data['P2_E4_no_compression'] = experiment_data['P2_E4']
    print(f"P2_E4_no_compression: {len(experiment_data['P2_E4'])} runs")
    
if 'P3_E1' in experiment_data:
    config_data['P3_E1_carbon_optimized'] = experiment_data['P3_E1']
    print(f"P3_E1_carbon_optimized: {len(experiment_data['P3_E1'])} runs")

print(f"\nTotal configurations for analysis: {len(config_data)}")

## Statistical Comparison Framework

Implementing statistical comparison functions to analyze configuration effects against baseline with appropriate test selection based on data distribution characteristics.

In [ ]:
def statistical_comparison(config_data: pd.DataFrame, baseline_data: pd.DataFrame, 
                         metric: str = 'sci_score') -> Dict:
    config_values = config_data[metric].dropna()
    baseline_values = baseline_data[metric].dropna()
    
    if len(config_values) < 3 or len(baseline_values) < 3:
        return {'error': 'Insufficient data'}
    
    config_normal = shapiro(config_values)[1] > 0.05
    baseline_normal = shapiro(baseline_values)[1] > 0.05
    
    if config_normal and baseline_normal:
        equal_var = levene(config_values, baseline_values)[1] > 0.05
        stat, p_value = ttest_ind(config_values, baseline_values, equal_var=equal_var)
        test_type = f"t-test ({'equal' if equal_var else 'unequal'} var)"
        
        pooled_std = np.sqrt(((len(config_values)-1)*config_values.var() + 
                             (len(baseline_values)-1)*baseline_values.var()) / 
                            (len(config_values)+len(baseline_values)-2))
        effect_size = (baseline_values.mean() - config_values.mean()) / pooled_std
    else:
        stat, p_value = mannwhitneyu(config_values, baseline_values, alternative='two-sided')
        test_type = "Mann-Whitney U"
        n1, n2 = len(baseline_values), len(config_values)
        cliff_delta = (sum(x > y for x in baseline_values for y in config_values) - 
                      sum(x < y for x in baseline_values for y in config_values)) / (n1 * n2)
        effect_size = cliff_delta
    
    improvement = ((baseline_values.mean() - config_values.mean()) / 
                  baseline_values.mean()) * 100
    
    return {
        'config_mean': config_values.mean(),
        'config_std': config_values.std(),
        'baseline_mean': baseline_values.mean(),
        'baseline_std': baseline_values.std(),
        'improvement_pct': improvement,
        'p_value': p_value,
        'effect_size': effect_size,
        'test_type': test_type,
        'significant': p_value < 0.05,
        'config_n': len(config_values),
        'baseline_n': len(baseline_values)
    }

rq2_results = {}
metrics = ['sci_score', 'powerjoular_power', 'test_duration', 'powerjoular_util']

for config_name, config_df in config_data.items():
    rq2_results[config_name] = {}
    
    for metric in metrics:
        if metric in config_df.columns and metric in baseline_data.columns:
            result = statistical_comparison(config_df, baseline_data, metric)
            rq2_results[config_name][metric] = result

print("Statistical analysis completed for all configurations")

## Multiple Comparison Correction

Implementing Bonferroni and Holm corrections for multiple comparisons to maintain statistical rigor.

In [ ]:
p_values_for_correction = []
test_descriptions = []

for config_name, metrics_data in rq2_results.items():
    for metric, result in metrics_data.items():
        if not result.get('error') and 'p_value' in result:
            p_values_for_correction.append(result['p_value'])
            test_descriptions.append(f"{config_name}_{metric}")

if p_values_for_correction:
    bonferroni_rejected, bonferroni_corrected, _, bonferroni_alpha = multipletests(
        p_values_for_correction, alpha=0.05, method='bonferroni'
    )
    
    holm_rejected, holm_corrected, _, holm_alpha = multipletests(
        p_values_for_correction, alpha=0.05, method='holm'
    )
    
    correction_results = pd.DataFrame({
        'Test': test_descriptions,
        'Original_P_Value': p_values_for_correction,
        'Bonferroni_Corrected_P': bonferroni_corrected,
        'Bonferroni_Significant': bonferroni_rejected,
        'Holm_Corrected_P': holm_corrected,
        'Holm_Significant': holm_rejected
    })
    
    correction_results.to_csv(results_dir / 'multiple_comparison_corrections.csv', index=False)
    
    print(f"Multiple Comparison Correction Applied:")
    print(f"Total tests: {len(p_values_for_correction)}")
    print(f"Original alpha: 0.05")
    print(f"Bonferroni alpha: {bonferroni_alpha:.6f}")
    print(f"Significant after Bonferroni: {sum(bonferroni_rejected)}")
    print(f"Significant after Holm: {sum(holm_rejected)}")
    
    significant_after_correction = correction_results[
        correction_results['Holm_Significant'] == True
    ].sort_values('Holm_Corrected_P')
    
    if len(significant_after_correction) > 0:
        print(f"\nSignificant results after Holm correction:")
        for _, row in significant_after_correction.head(10).iterrows():
            print(f"  {row['Test']}: p = {row['Holm_Corrected_P']:.6f}")
    else:
        print("\nNo results remain significant after multiple comparison correction.")
else:
    print("No p-values available for multiple comparison correction.")

## Bootstrap Confidence Intervals

Using bootstrap resampling to estimate confidence intervals for effect sizes and validate the robustness of our findings.

In [ ]:
def bootstrap_effect_size(group1, group2, n_bootstrap=1000, confidence_level=0.95):

    def cohen_d(x, y):
        """Calculate Cohen's d effect size"""
        pooled_std = np.sqrt(((len(x) - 1) * np.var(x, ddof=1) + 
                             (len(y) - 1) * np.var(y, ddof=1)) / 
                            (len(x) + len(y) - 2))
        return (np.mean(x) - np.mean(y)) / pooled_std
    
    bootstrap_effects = []
    for _ in range(n_bootstrap):
        boot_group1 = np.random.choice(group1, size=len(group1), replace=True)
        boot_group2 = np.random.choice(group2, size=len(group2), replace=True)
        
        effect = cohen_d(boot_group1, boot_group2)
        bootstrap_effects.append(effect)
    
    alpha = 1 - confidence_level
    lower_percentile = (alpha / 2) * 100
    upper_percentile = (1 - alpha / 2) * 100
    
    ci_lower = np.percentile(bootstrap_effects, lower_percentile)
    ci_upper = np.percentile(bootstrap_effects, upper_percentile)
    
    return {
        'effect_size': cohen_d(group1, group2),
        'bootstrap_mean': np.mean(bootstrap_effects),
        'bootstrap_std': np.std(bootstrap_effects),
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'bootstrap_effects': bootstrap_effects
    }

bootstrap_results = []

for config_name, config_df in config_data.items():
    if 'sci_score' in config_df.columns:
        config_sci = config_df['sci_score'].dropna()
        baseline_sci = baseline_data['sci_score'].dropna()
        
        if len(config_sci) >= 3 and len(baseline_sci) >= 3:
            boot_result = bootstrap_effect_size(baseline_sci, config_sci)
            
            ci_significant = not (boot_result['ci_lower'] <= 0 <= boot_result['ci_upper'])
            
            bootstrap_results.append({
                'Configuration': config_name,
                'Effect_Size': boot_result['effect_size'],
                'Bootstrap_Mean': boot_result['bootstrap_mean'],
                'Bootstrap_Std': boot_result['bootstrap_std'],
                'CI_Lower': boot_result['ci_lower'],
                'CI_Upper': boot_result['ci_upper'],
                'CI_Significant': ci_significant
            })

bootstrap_df = pd.DataFrame(bootstrap_results)
if not bootstrap_df.empty:
    bootstrap_df = bootstrap_df.sort_values('Effect_Size', ascending=False)
    bootstrap_df.to_csv(results_dir / 'bootstrap_effect_sizes.csv', index=False)
    
    print("Bootstrap Effect Size Analysis (SCI Score):")
    print("=" * 50)
    
    for _, row in bootstrap_df.head(10).iterrows():
        significance = "***" if row['CI_Significant'] else "n.s."
        print(f"{row['Configuration'][:25]:25} Effect: {row['Effect_Size']:6.3f} "
              f"95% CI: [{row['CI_Lower']:6.3f}, {row['CI_Upper']:6.3f}] {significance}")
    
    robust_significant = bootstrap_df[bootstrap_df['CI_Significant'] == True]
    print(f"\nConfigurations with robust significant effects: {len(robust_significant)}")
    
else:
    print("No bootstrap results generated.")

## Power Analysis and Sample Size Assessment

Conducting post-hoc power analysis to assess the adequacy of sample sizes and identify potential Type II errors.

In [ ]:
power_analysis_results = []

for exp_id, exp_df in experiment_data.items():
    if exp_id != 'P1_E1':
        exp_sci = exp_df['sci_score'].dropna()
        baseline_sci = baseline_data['sci_score'].dropna()
        
        if len(exp_sci) >= 3 and len(baseline_sci) >= 3:
            pooled_std = np.sqrt(((len(exp_sci) - 1) * exp_sci.var() + 
                                 (len(baseline_sci) - 1) * baseline_sci.var()) / 
                                (len(exp_sci) + len(baseline_sci) - 2))
            
            observed_effect = abs((baseline_sci.mean() - exp_sci.mean()) / pooled_std)
            
            achieved_power = ttest_power(
                effect_size=observed_effect,
                nobs=min(len(exp_sci), len(baseline_sci)),
                alpha=0.05,
                alternative='two-sided'
            )
            
            required_n_80 = int(np.ceil(tt_solve_power(
                effect_size=observed_effect,
                power=0.8,
                alpha=0.05,
                alternative='two-sided'
            ))) if observed_effect > 0 else np.inf
            
            required_n_90 = int(np.ceil(tt_solve_power(
                effect_size=observed_effect,
                power=0.9,
                alpha=0.05,
                alternative='two-sided'
            ))) if observed_effect > 0 else np.inf
            
            power_analysis_results.append({
                'Experiment': exp_id,
                'Sample_Size': len(exp_sci),
                'Observed_Effect_Size': observed_effect,
                'Achieved_Power': achieved_power,
                'Required_N_80_Power': required_n_80,
                'Required_N_90_Power': required_n_90,
                'Adequate_Power_80': achieved_power >= 0.8,
                'Adequate_Power_90': achieved_power >= 0.9
            })

power_df = pd.DataFrame(power_analysis_results)
if not power_df.empty:
    power_df.to_csv(results_dir / 'power_analysis_results.csv', index=False)
    
    print("Post-hoc Power Analysis Results:")
    print("=" * 50)
    print(f"{'Experiment':<12} {'N':<4} {'Effect':<8} {'Power':<8} {'80% Power':<11} {'90% Power':<11}")
    print("-" * 65)
    
    for _, row in power_df.iterrows():
        power_80_status = "✓" if row['Adequate_Power_80'] else "✗"
        power_90_status = "✓" if row['Adequate_Power_90'] else "✗"
        
        print(f"{row['Experiment']:<12} {row['Sample_Size']:<4} "
              f"{row['Observed_Effect_Size']:<8.3f} {row['Achieved_Power']:<8.3f} "
              f"{power_80_status:<11} {power_90_status:<11}")
    
    adequate_80 = power_df['Adequate_Power_80'].sum()
    adequate_90 = power_df['Adequate_Power_90'].sum()
    total_exp = len(power_df)
    
    print(f"\nPower Analysis Summary:")
    print(f"Experiments with adequate power (≥80%): {adequate_80}/{total_exp} ({adequate_80/total_exp*100:.1f}%)")
    print(f"Experiments with high power (≥90%): {adequate_90}/{total_exp} ({adequate_90/total_exp*100:.1f}%)")
    
    low_power = power_df[power_df['Achieved_Power'] < 0.8]
    if len(low_power) > 0:
        print(f"\nExperiments with potentially inadequate power (risk of Type II error):")
        for _, row in low_power.iterrows():
            print(f"   {row['Experiment']}: Power = {row['Achieved_Power']:.3f}, "
                  f"Recommend n ≥ {row['Required_N_80_Power']} for 80% power")
else:
    print("No power analysis results generated.")

## Impact of Application-Level Configuration Settings

Analyzing how different configuration parameters affect carbon emissions compared to the baseline (P1_E1).

In [ ]:
print(f"Baseline SCI: {baseline_sci_mean:.3f} ± {baseline_sci_std:.3f} gCO2eq/scenario (n={len(baseline_data)})")
print("\nRQ2 Analysis: Comparing configurations against baseline\n")

rq2_comparison_results = []

for exp_id in ['P2_E1', 'P2_E2', 'P2_E3', 'P2_E4', 'P2_E5']:
    if exp_id in experiment_data:
        exp_data = experiment_data[exp_id]
        factor_col = experiments[exp_id].get('factor')
        
        print(f"\n{experiments[exp_id]['name']} ({exp_id}):")
        print("-" * 50)
        
        if factor_col and factor_col in exp_data.columns:
            configurations = exp_data[factor_col].unique()
            
            for config in configurations:
                config_data_subset = exp_data[exp_data[factor_col] == config]
                config_sci_mean = config_data_subset['sci_score'].mean()
                config_sci_std = config_data_subset['sci_score'].std()
                
                improvement = ((baseline_sci_mean - config_sci_mean) / baseline_sci_mean) * 100
                
                stat, p_value = mannwhitneyu(baseline_data['sci_score'], config_data_subset['sci_score'], alternative='two-sided')
                
                n1, n2 = len(baseline_data), len(config_data_subset)
                cliff_delta = (sum(x > y for x in baseline_data['sci_score'] for y in config_data_subset['sci_score']) - 
                             sum(x < y for x in baseline_data['sci_score'] for y in config_data_subset['sci_score'])) / (n1 * n2)
                
                significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "n.s."
                
                print(f"  {config}: {config_sci_mean:.3f} ± {config_sci_std:.3f} (n={len(config_data_subset)})")
                print(f"    Improvement: {improvement:+.1f}% | p={p_value:.4f} {significance} | δ={cliff_delta:.3f}")
                
                rq2_comparison_results.append({
                    'experiment': exp_id,
                    'experiment_name': experiments[exp_id]['name'],
                    'configuration': config,
                    'baseline_mean': baseline_sci_mean,
                    'config_mean': config_sci_mean,
                    'improvement_percent': improvement,
                    'p_value': p_value,
                    'cliff_delta': cliff_delta,
                    'significant': p_value < 0.05,
                    'n_baseline': len(baseline_data),
                    'n_config': len(config_data_subset)
                })
        else:
            exp_sci_mean = exp_data['sci_score'].mean()
            exp_sci_std = exp_data['sci_score'].std()
            improvement = ((baseline_sci_mean - exp_sci_mean) / baseline_sci_mean) * 100
            
            stat, p_value = mannwhitneyu(baseline_data['sci_score'], exp_data['sci_score'], alternative='two-sided')
            
            significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "n.s."
            print(f"  Overall: {exp_sci_mean:.3f} ± {exp_sci_std:.3f} (n={len(exp_data)})")
            print(f"    Improvement: {improvement:+.1f}% | p={p_value:.4f} {significance}")

rq2_df = pd.DataFrame(rq2_comparison_results)
if not rq2_df.empty:
    rq2_df.to_csv(results_dir / 'rq2_statistical_analysis.csv', index=False)

    significant_improvements = rq2_df[rq2_df['significant'] & (rq2_df['improvement_percent'] > 0)]
    print(f"\nRQ2 Summary: {len(significant_improvements)} out of {len(rq2_df)} configurations show significant improvement")
    if len(rq2_df) > 0:
        print(f"Best improvement: {rq2_df['improvement_percent'].max():.1f}% (p={rq2_df.loc[rq2_df['improvement_percent'].idxmax(), 'p_value']:.4f})")

## Statistical Analysis Results

Analyzing significance and effect sizes across all configuration metrics.

In [ ]:
significant_results = []

for config_name, metrics_data in rq2_results.items():
    sci_result = metrics_data.get('sci_score', {})
    if sci_result.get('significant', False) and not sci_result.get('error'):
        significant_results.append({
            'Configuration': config_name,
            'SCI_Mean': sci_result['config_mean'],
            'SCI_Std': sci_result['config_std'],
            'Improvement_%': sci_result['improvement_pct'],
            'P_Value': sci_result['p_value'],
            'Effect_Size': sci_result['effect_size'],
            'Test_Type': sci_result['test_type'],
            'Sample_Size': sci_result['config_n']
        })

significant_df = pd.DataFrame(significant_results)
if not significant_df.empty:
    significant_df = significant_df.sort_values('Improvement_%', ascending=False)
    significant_df.to_csv(results_dir / 'rq2_significant_configurations.csv', index=False)
    
    print("Significant SCI Improvements vs Baseline:")
    print(significant_df[['Configuration', 'Improvement_%', 'P_Value']].to_string(index=False))
else:
    print("No statistically significant SCI improvements found")

## SCI Distribution Analysis by Configuration

Visualizing the distribution of SCI scores across different configurations using violin plots to show central tendency and distribution shape.

In [ ]:
plot_data = []

for _, row in baseline_data.iterrows():
    plot_data.append({
        'SCI_Score': row['sci_score'],
        'Configuration': 'Baseline',
        'Experiment': 'P1_E1'
    })

for exp_id in ['P2_E1', 'P2_E2', 'P2_E3', 'P2_E4', 'P2_E5']:
    if exp_id in experiment_data:
        exp_data = experiment_data[exp_id]
        factor_col = experiments[exp_id].get('factor')

        if factor_col and factor_col in exp_data.columns:
            for _, row in exp_data.iterrows():
                plot_data.append({
                    'SCI_Score': row['sci_score'],
                    'Configuration': row[factor_col],
                    'Experiment': exp_id
                })

if 'P3_E1' in experiment_data:
    for _, row in experiment_data['P3_E1'].iterrows():
        plot_data.append({
            'SCI_Score': row['sci_score'],
            'Configuration': 'Carbon-Optimized',
            'Experiment': 'P3_E1'
        })

plot_df = pd.DataFrame(plot_data)

fig, ax = plt.subplots(figsize=(16, 10))

experiment_order = ['P1_E1', 'P2_E1', 'P2_E2', 'P2_E3', 'P2_E4', 'P2_E5', 'P3_E1']

config_to_experiment = plot_df.set_index('Configuration')['Experiment'].to_dict()

unique_configs = plot_df['Configuration'].unique()
config_order = sorted(unique_configs, key=lambda c: experiment_order.index(config_to_experiment[c]))

experiment_palette = sns.color_palette('tab10', len(experiment_order))
experiment_color_map = dict(zip(experiment_order, experiment_palette))

violin_plot = sns.violinplot(
    data=plot_df,
    x='Configuration',
    y='SCI_Score',
    order=config_order,
    inner='box',
    ax=ax
)

ax.set_xticklabels(config_order, rotation=45)
ax.set_xlabel('Configuration', fontsize=14)
ax.set_ylabel('SCI Score (gCO₂e/scenario)', fontsize=14)

ax.axhline(y=baseline_sci_mean, color='red', linestyle='--', alpha=0.7,
           label=f'Baseline Mean: {baseline_sci_mean:.3f}')

for i, config in enumerate(config_order):
    experiment = config_to_experiment[config]
    color = experiment_color_map[experiment]
    violin_plot.collections[i * 2].set_facecolor(color)

legend_patches = [mpatches.Patch(color=color, label=exp) for exp, color in experiment_color_map.items()]
ax.legend(handles=legend_patches, title='Experiment', bbox_to_anchor=(1.01, 1), loc='upper left')

plt.tight_layout()
plt.savefig(figures_dir / 'sci_distribution_by_configuration.png')
plt.show()

print(f"Violin plot saved to {figures_dir / 'sci_distribution_by_configuration.png'}")

## Configuration Effect Heatmap

Creating a heatmap showing the effect sizes and significance levels of different configurations across multiple metrics.

In [ ]:
heatmap_data = []
metrics_display = {
    'sci_score': 'SCI Score',
    'powerjoular_power': 'Power (W)',
    'test_duration': 'Duration (s)',
    'powerjoular_util': 'CPU Usage (%)'
}

for config_name, metrics_data in rq2_results.items():
    for metric, metric_display in metrics_display.items():
        result = metrics_data.get(metric, {})
        if not result.get('error'):
            heatmap_data.append({
                'Configuration': config_name.replace('_', ' '),
                'Metric': metric_display,
                'Improvement_%': result.get('improvement_pct', 0),
                'Effect_Size': abs(result.get('effect_size', 0)),
                'Significant': result.get('significant', False),
                'P_Value': result.get('p_value', 1.0)
            })

heatmap_df = pd.DataFrame(heatmap_data)

if not heatmap_df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    
    pivot_improvement = heatmap_df.pivot(index='Configuration', columns='Metric', values='Improvement_%')
    sns.heatmap(pivot_improvement, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
                ax=ax1, cbar_kws={'label': 'Improvement %'})
    ax1.collections[0].colorbar.ax.set_ylabel('Improvement %', fontsize=14)
    ax1.set_xlabel('Metrics', fontweight='bold', fontsize=14)
    ax1.set_ylabel('Configuration', fontweight='bold', fontsize=14)
    
    pivot_sig = heatmap_df.pivot(index='Configuration', columns='Metric', values='Significant')
    for i in range(len(pivot_improvement.index)):
        for j in range(len(pivot_improvement.columns)):
            if pivot_sig.iloc[i, j]:
                ax1.add_patch(Rectangle((j, i), 1, 1, fill=False, edgecolor='black', lw=3))
    
    pivot_effect = heatmap_df.pivot(index='Configuration', columns='Metric', values='Effect_Size')
    sns.heatmap(pivot_effect, annot=True, fmt='.2f', cmap='viridis',
                ax=ax2, cbar_kws={'label': 'Effect Size (absolute)'})
    ax2.collections[0].colorbar.ax.set_ylabel('Effect Size (absolute)', fontsize=14)
    ax2.set_xlabel('Metrics', fontweight='bold', fontsize=14)
    ax2.set_ylabel('')

    ax1.tick_params(axis='x', labelsize=12)
    ax1.tick_params(axis='y', labelsize=12)
    
    ax2.tick_params(axis='x', labelsize=12)
    ax2.tick_params(axis='y', labelsize=12)

    
    for i in range(len(pivot_effect.index)):
        for j in range(len(pivot_effect.columns)):
            if pivot_sig.iloc[i, j]:
                ax2.add_patch(Rectangle((j, i), 1, 1, fill=False, edgecolor='white', lw=3))
    
    plt.tight_layout()
    plt.savefig(figures_dir / 'rq2_configuration_effects_heatmap.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    heatmap_df.to_csv(results_dir / 'rq2_all_configuration_effects.csv', index=False)
    print("Configuration effects heatmap saved")
else:
    print("Insufficient data for heatmap visualization")

## Resource Utilization vs SCI Analysis

Examining the relationship between system resource utilization and carbon intensity.

In [ ]:
resource_data = []

for exp_id, exp_df in experiment_data.items():
    for _, row in exp_df.iterrows():
        if pd.notna(row.get('vm_avg_cpu_percent')) and pd.notna(row.get('vm_total_io_read_count')):
            resource_data.append({
                'SCI_Score': row['sci_score'],
                'CPU_Utilization': row['vm_avg_cpu_percent'],
                'IO_Read_Operations': row['vm_total_io_read_count'] / 1e6,
                'Experiment': exp_id,
                'Category': experiments[exp_id]['category'],
                'test_duration': row['test_duration']
            })

resource_df = pd.DataFrame(resource_data)

if not resource_df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), gridspec_kw={'width_ratios': [1, 1]})
    
    scatter1 = ax1.scatter(
        resource_df['CPU_Utilization'], resource_df['SCI_Score'],
        c=resource_df['test_duration'], s=60, alpha=0.7,
        cmap='plasma', edgecolors='black', linewidth=0.5
    )
    ax1.set_xlabel('CPU Utilization (%)', fontsize=12)
    ax1.set_ylabel('SCI Score (gCO₂e/scenario)', fontsize=12)
    
    corr_cpu = resource_df['CPU_Utilization'].corr(resource_df['SCI_Score'])
    ax1.text(0.05, 0.95, f'r = {corr_cpu:.3f}', transform=ax1.transAxes,
             fontsize=12, fontweight='bold', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    scatter2 = ax2.scatter(
        resource_df['IO_Read_Operations'], resource_df['SCI_Score'],
        c=resource_df['test_duration'], s=60, alpha=0.7,
        cmap='plasma', edgecolors='black', linewidth=0.5
    )
    ax2.set_xlabel('I/O Read Operations (millions)', fontsize=12)
    ax2.set_ylabel('SCI Score (gCO₂e/scenario)', fontsize=12)
    
    corr_io = resource_df['IO_Read_Operations'].corr(resource_df['SCI_Score'])
    ax2.text(0.05, 0.95, f'r = {corr_io:.3f}', transform=ax2.transAxes,
             fontsize=12, fontweight='bold', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    cbar_ax = fig.add_axes([0.92, 0.15, 0.015, 0.7])
    cbar = fig.colorbar(scatter1, cax=cbar_ax)
    cbar.set_label('Duration (s)', rotation=270, labelpad=20)
    
    plt.subplots_adjust(wspace=0.3)
    plt.savefig(figures_dir / 'resource_utilization_vs_sci.png')
    plt.show()
    
    correlation_results = pd.DataFrame({
        'Metric_Pair': ['SCI_vs_CPU', 'SCI_vs_IO_Read', 'Duration_vs_CPU', 'Duration_vs_IO_Read'],
        'Correlation': [
            corr_cpu,
            corr_io,
            resource_df['test_duration'].corr(resource_df['CPU_Utilization']),
            resource_df['test_duration'].corr(resource_df['IO_Read_Operations'])
        ]
    })
    correlation_results.to_csv(results_dir / 'resource_correlations.csv', index=False)
    
    print(f"Resource utilization plot saved to {figures_dir / 'resource_utilization_vs_sci.png'}")
    print(f"Correlation analysis saved to {results_dir / 'resource_correlations.csv'}")
else:
    print("Insufficient resource utilization data for analysis")

## Partial Correlation Analysis

Examining relationships between variables while controlling for potential confounders to identify causal relationships.

In [ ]:
def partial_correlation(x, y, control_vars, data):
    from scipy.stats import pearsonr
    from sklearn.linear_model import LinearRegression
    
    clean_data = data[[x, y] + control_vars].dropna()
    
    if len(clean_data) < 10:
        return None, None
    
    X_controls = clean_data[control_vars]
    reg_x = LinearRegression().fit(X_controls, clean_data[x])
    x_residuals = clean_data[x] - reg_x.predict(X_controls)
    
    reg_y = LinearRegression().fit(X_controls, clean_data[y])
    y_residuals = clean_data[y] - reg_y.predict(X_controls)
    
    r, p_value = pearsonr(x_residuals, y_residuals)
    
    return r, p_value

partial_correlations = []

relationships = [
    ('sci_score', 'powerjoular_power', ['test_duration']),
    ('sci_score', 'powerjoular_util', ['test_duration']),
    ('sci_score', 'vm_avg_memory_mb', ['test_duration', 'powerjoular_util']),
    ('powerjoular_power', 'powerjoular_util', ['test_duration']),
    ('test_duration', 'powerjoular_util', ['powerjoular_power'])
]

print("Partial Correlation Analysis:")
print("=" * 50)
print(f"{'Relationship':<35} {'Zero-order r':<12} {'Partial r':<12} {'p-value':<10}")
print("-" * 75)

for var1, var2, controls in relationships:
    clean_data = all_data[[var1, var2]].dropna()
    if len(clean_data) > 0:
        zero_order_r, _ = pearsonr(clean_data[var1], clean_data[var2])
        
        partial_r, partial_p = partial_correlation(var1, var2, controls, all_data)
        
        if partial_r is not None:
            relationship_name = f"{var1} ↔ {var2}"
            controls_str = f"(controlling for {', '.join(controls)})"
            
            print(f"{relationship_name:<35} {zero_order_r:<12.3f} {partial_r:<12.3f} {partial_p:<10.3f}")
            
            partial_correlations.append({
                'Variable_1': var1,
                'Variable_2': var2,
                'Control_Variables': ', '.join(controls),
                'Zero_Order_r': zero_order_r,
                'Partial_r': partial_r,
                'P_Value': partial_p,
                'Significant': partial_p < 0.05 if partial_p is not None else False
            })

if partial_correlations:
    partial_corr_df = pd.DataFrame(partial_correlations)
    partial_corr_df.to_csv(results_dir / 'partial_correlation_analysis.csv', index=False)
    
    print(f"\nPartial correlation analysis saved to {results_dir / 'partial_correlation_analysis.csv'}")
    
    print("\nKey findings:")
    for _, row in partial_corr_df.iterrows():
        diff = abs(row['Zero_Order_r'] - row['Partial_r'])
        if diff > 0.2:
            print(f"  {row['Variable_1']} ↔ {row['Variable_2']}: "
                  f"r changes from {row['Zero_Order_r']:.3f} to {row['Partial_r']:.3f} "
                  f"when controlling for {row['Control_Variables']}")
else:
    print("No partial correlation results generated.")

## Carbon-Optimized Configuration Effectiveness

Evaluating the combined carbon-optimized configuration (P3_E1) against the baseline to test H3.1.

In [ ]:
if 'P3_E1' in experiment_data:
    carbon_optimized_data = experiment_data['P3_E1']
    
    baseline_sci = baseline_data['sci_score']
    optimized_sci = carbon_optimized_data['sci_score']
    
    baseline_mean = baseline_sci.mean()
    baseline_std = baseline_sci.std()
    optimized_mean = optimized_sci.mean()
    optimized_std = optimized_sci.std()
    
    improvement = ((baseline_mean - optimized_mean) / baseline_mean) * 100
    
    stat, p_value = mannwhitneyu(baseline_sci, optimized_sci, alternative='two-sided')
    
    pooled_std = np.sqrt(((len(baseline_sci)-1)*baseline_sci.var() + 
                         (len(optimized_sci)-1)*optimized_sci.var()) / 
                        (len(baseline_sci)+len(optimized_sci)-2))
    cohens_d = (baseline_mean - optimized_mean) / pooled_std
    
    diff_mean = baseline_mean - optimized_mean
    diff_std = np.sqrt(baseline_std**2/len(baseline_sci) + optimized_std**2/len(optimized_sci))
    ci_lower = diff_mean - 1.96 * diff_std
    ci_upper = diff_mean + 1.96 * diff_std
    
    print("RQ3 Analysis: Carbon-Optimized Configuration Effectiveness")
    print("=" * 60)
    print(f"Baseline (P1_E1): {baseline_mean:.3f} ± {baseline_std:.3f} gCO2eq/scenario (n={len(baseline_sci)})")
    print(f"Carbon-Optimized (P3_E1): {optimized_mean:.3f} ± {optimized_std:.3f} gCO2eq/scenario (n={len(optimized_sci)})")
    print(f"\nImprovement: {improvement:.1f}%")
    print(f"Absolute reduction: {diff_mean:.3f} gCO2eq/scenario")
    print(f"95% CI for difference: [{ci_lower:.3f}, {ci_upper:.3f}]")
    print(f"\nStatistical significance: p = {p_value:.4f}")
    print(f"Effect size (Cohen's d): {cohens_d:.3f}")
    
    if p_value < 0.05:
        print("\nConclusion: H3.0 is REJECTED. Carbon-optimized configuration significantly reduces SCI.")
    else:
        print("\nConclusion: H3.0 is NOT rejected. No significant reduction observed.")
    
    rq3_results = pd.DataFrame({
        'Metric': ['Baseline_Mean', 'Baseline_Std', 'Optimized_Mean', 'Optimized_Std', 
                  'Improvement_Percent', 'P_Value', 'Cohens_D', 'CI_Lower', 'CI_Upper'],
        'Value': [baseline_mean, baseline_std, optimized_mean, optimized_std, 
                 improvement, p_value, cohens_d, ci_lower, ci_upper]
    })
    rq3_results.to_csv(results_dir / 'rq3_statistical_analysis.csv', index=False)
    
else:
    print("P3_E1 data not available for RQ3 analysis")
    improvement = None
    p_value = None
    cohens_d = None

## Statistical Summary Tables

Generating statistical summaries for inclusion in the thesis report.

In [ ]:
summary_stats = []

baseline_stats = {
    'Configuration': 'P1_E1 Baseline',
    'N': len(baseline_data),
    'Mean_SCI': baseline_data['sci_score'].mean(),
    'Std_SCI': baseline_data['sci_score'].std(),
    'Median_SCI': baseline_data['sci_score'].median(),
    'Min_SCI': baseline_data['sci_score'].min(),
    'Max_SCI': baseline_data['sci_score'].max(),
    'Mean_Power': baseline_data['powerjoular_power'].mean(),
    'Mean_Duration': baseline_data['test_duration'].mean(),
    'Success_Rate': baseline_data['test_success_rate'].mean()
}
summary_stats.append(baseline_stats)

for exp_id in ['P2_E1', 'P2_E2', 'P2_E3', 'P2_E4', 'P2_E5']:
    if exp_id in experiment_data:
        exp_data = experiment_data[exp_id]
        factor_col = experiments[exp_id].get('factor')
        
        if factor_col and factor_col in exp_data.columns:
            for config in exp_data[factor_col].unique():
                config_data_subset = exp_data[exp_data[factor_col] == config]
                
                config_stats = {
                    'Configuration': f"{exp_id}_{config}",
                    'N': len(config_data_subset),
                    'Mean_SCI': config_data_subset['sci_score'].mean(),
                    'Std_SCI': config_data_subset['sci_score'].std(),
                    'Median_SCI': config_data_subset['sci_score'].median(),
                    'Min_SCI': config_data_subset['sci_score'].min(),
                    'Max_SCI': config_data_subset['sci_score'].max(),
                    'Mean_Power': config_data_subset['powerjoular_power'].mean(),
                    'Mean_Duration': config_data_subset['test_duration'].mean(),
                    'Success_Rate': config_data_subset['test_success_rate'].mean()
                }
                summary_stats.append(config_stats)
        else:
            config_stats = {
                'Configuration': f"{exp_id}_no_compression",
                'N': len(exp_data),
                'Mean_SCI': exp_data['sci_score'].mean(),
                'Std_SCI': exp_data['sci_score'].std(),
                'Median_SCI': exp_data['sci_score'].median(),
                'Min_SCI': exp_data['sci_score'].min(),
                'Max_SCI': exp_data['sci_score'].max(),
                'Mean_Power': exp_data['powerjoular_power'].mean(),
                'Mean_Duration': exp_data['test_duration'].mean(),
                'Success_Rate': exp_data['test_success_rate'].mean()
            }
            summary_stats.append(config_stats)

if 'P3_E1' in experiment_data:
    p3_data = experiment_data['P3_E1']
    p3_stats = {
        'Configuration': 'P3_E1 Carbon-Optimized',
        'N': len(p3_data),
        'Mean_SCI': p3_data['sci_score'].mean(),
        'Std_SCI': p3_data['sci_score'].std(),
        'Median_SCI': p3_data['sci_score'].median(),
        'Min_SCI': p3_data['sci_score'].min(),
        'Max_SCI': p3_data['sci_score'].max(),
        'Mean_Power': p3_data['powerjoular_power'].mean(),
        'Mean_Duration': p3_data['test_duration'].mean(),
        'Success_Rate': p3_data['test_success_rate'].mean()
    }
    summary_stats.append(p3_stats)

summary_df = pd.DataFrame(summary_stats)
if len(summary_df) > 0:
    summary_df['Improvement_vs_Baseline'] = ((summary_df.iloc[0]['Mean_SCI'] - summary_df['Mean_SCI']) / 
                                             summary_df.iloc[0]['Mean_SCI']) * 100

    summary_df.to_csv(results_dir / 'comprehensive_statistical_summary.csv', index=False)
    
    print("Top 5 Performing Configurations (by SCI reduction):")
    print("=" * 55)
    top_configs = summary_df.nlargest(5, 'Improvement_vs_Baseline')
    for _, row in top_configs.iterrows():
        print(f"{row['Configuration']}: {row['Mean_SCI']:.3f} gCO2eq/scenario ({row['Improvement_vs_Baseline']:+.1f}%)")
    
    print(f"\nComprehensive summary saved to {results_dir / 'comprehensive_statistical_summary.csv'}")

## Hypothesis Testing Summary

Providing definitive conclusions for both research questions based on statistical evidence.

In [ ]:
print("HYPOTHESIS TESTING RESULTS")
print("=" * 50)

if not rq2_df.empty:
    significant_configs = rq2_df[rq2_df['significant'] & (rq2_df['improvement_percent'] > 0)]
    print(f"\nRQ2 - Impact of Application-Level Configurations:")
    print(f"Configurations tested: {len(rq2_df)}")
    print(f"Significant improvements: {len(significant_configs)}")
    print(f"Best improvement: {rq2_df['improvement_percent'].max():.1f}%")
    
    if len(significant_configs) > 0:
        print("\nH1.0 REJECTED: At least one configuration significantly affects SCI score")
        print("H1.a ACCEPTED: Configuration changes can significantly reduce carbon emissions")
        
        print("\nSignificant configurations:")
        for _, row in significant_configs.iterrows():
            print(f"  {row['experiment']}_{row['configuration']}: {row['improvement_percent']:+.1f}% (p={row['p_value']:.4f})")
    else:
        print("\nH1.0 NOT REJECTED: No significant configuration effects detected")
        print("H1.a REJECTED: Configuration changes do not significantly affect SCI score")
else:
    print("\nRQ2 - No comparison data available")
    significant_configs = pd.DataFrame()

if 'P3_E1' in experiment_data and improvement is not None:
    print(f"\n\nRQ3 - Carbon-Optimized Configuration Effectiveness:")
    print(f"Improvement achieved: {improvement:.1f}%")
    print(f"Statistical significance: p = {p_value:.4f}")
    print(f"Effect size (Cohen's d): {cohens_d:.3f}")
    
    if p_value < 0.05 and improvement > 0:
        print("\nH2.0 REJECTED: The combined optimal SCI configuration significantly affects SCI score")
        print("H2.a ACCEPTED: The carbon-optimized configuration has a significant effect")
        
        if cohens_d >= 0.8:
            effect_interpretation = "large"
        elif cohens_d >= 0.5:
            effect_interpretation = "medium"
        elif cohens_d >= 0.2:
            effect_interpretation = "small"
        else:
            effect_interpretation = "negligible"
        
        print(f"Effect size interpretation: {effect_interpretation} practical significance")
    else:
        print("\nH2.0 NOT REJECTED: No significant effect from the combined optimal SCI configuration")
        print("H2.a REJECTED: The carbon-optimized configuration does not have a significant effect")
else:
    print("\n\nRQ3 - Carbon-Optimized configuration data not available")

hypothesis_results = {
    'RQ2_H10_Status': 'Rejected' if not rq2_df.empty and len(significant_configs) > 0 else 'Not Rejected',
    'RQ2_H1a_Status': 'Accepted' if not rq2_df.empty and len(significant_configs) > 0 else 'Rejected',
    'RQ2_Significant_Configs': len(significant_configs) if not rq2_df.empty else 0,
    'RQ2_Best_Improvement': rq2_df['improvement_percent'].max() if not rq2_df.empty else None,
    'RQ3_H20_Status': 'Rejected' if (improvement is not None and p_value < 0.05 and improvement > 0) else 'Not Rejected',
    'RQ3_H2a_Status': 'Accepted' if (improvement is not None and p_value < 0.05 and improvement > 0) else 'Rejected',
    'RQ3_Improvement_Percent': improvement,
    'RQ3_P_Value': p_value,
    'RQ3_Effect_Size': cohens_d
}

hypothesis_df = pd.DataFrame([hypothesis_results])
hypothesis_df.to_csv(results_dir / 'hypothesis_testing_results.csv', index=False)

print(f"\nHypothesis testing results saved to {results_dir / 'hypothesis_testing_results.csv'}")
print(f"\nAnalysis complete. All figures saved to {figures_dir}/")
print("\nGenerated files:")
for csv_file in results_dir.glob('*.csv'):
    print(f"- {csv_file.name}")
print("\nGenerated figures:")
for png_file in figures_dir.glob('*.png'):
    print(f"- {png_file.name}")